In [1]:
# Перечень команд, которые необходимо выполнить перед запуском jupyter notebook для включения решателей:
# export PATH=$PATH:/Library/Frameworks/GAMS.framework/Versions/47/Resources/
# export PATH=$PATH:/Library/Frameworks/ampl_macos64/

# Доступные решатели:
# _neos              [-] внутренняя ошибка связанная с работой со строками
# _mock_cbc          [-] попытка доступа к несуществующему полю _problem_files у ConcreteModel
# glpk               [-] не работает с нелинейными выражениями в целевой функции
# _glpk_shell        [-] не работает с нелинейными выражениями в целевой функции
# _mock_glpk         [-] попытка доступа к несуществующему полю _problem_files у ConcreteModel
# _mock_cplex        [-] не работает с нелинейными выражениями в целевой функции
# gurobi_direct      [-] не работает с нелинейными выражениями в целевой функции
# gurobi             [-] не работает с нелинейными выражениями в целевой функции
# _gurobi_shell      [-] не работает с нелинейными выражениями в целевой функции
# baron              [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# xpress             [-] не работает с нелинейными выражениями в целевой функции
# ipopt              [+] не работает с целыми значениями
# gurobi_persistent  [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# gams               [+]
# _gams_shell        [+]
# xpress_direct      [-] не работает с нелинейными выражениями в целевой функции
# xpress_persistent  [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# mpec_nlp           [+] не работает с целыми значениями
# mpec_minlp         [-] не работает с нелинейными выражениями в целевой функции
# appsi_gurobi       [-] не работает с нелинейными выражениями в целевой функции
# gdpopt             [-] требуется дополнительное указание алгоритма через аргумент метода solve         
# gdpopt.gloa        [-] ошибка инициализации решателя
# gdpopt.lbb         [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# gdpopt.loa         [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# gdpopt.ric         [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# gdpopt.enumerate   [-] не работает с нелинейными выражениями в целевой функции
# mindtpy            [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# mindtpy.oa         [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# mindtpy.ecp        [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# mindtpy.goa        [-] внутренняя ошибка работы решателя
# mindtpy.fp         [-] внутренняя ошибка при работе с сложными выражениями в целевой функции
# multistart         [+] не работает с целыми значениями
# ipopt_v2           [+] не работает с целыми значениями
# gurobi_v2          [-] не работает с нелинейными выражениями в целевой функции
# gurobi_direct_v2   [-] не работает с нелинейными выражениями в целевой функции
# trustregion        [-] для инициализации требуется дополнительный аргумент degrees_of_freedom_variables
# ampl               [-] внутренняя ошибка работы решателя

In [2]:
# Условные обозначения:
# Параметр              Размерность   Примечание
# n                                   - число требований
# m                                   - число файлов исходного кода
# k                                   - число плагинов
# М                                   - условно большое число
# T (traceability)        (n x m)     - матрица трассируемости требований к ПО на файлы исходного кода
# D (dependency)          (m x m)     - матрица зависимостей между файлами исходного кода
# P (price)               (n x n)     - матрица расчета стоимости сопровождения требований в поставке
# A (allocation)          (m x k)     - матрица распределения файлов исходного кода по плагинам
# useful_requirements     (1 х n)     - вектор полезных требованиях
# useful_files            (1 x m)     - вектор полезных файлов исходного кода 
#                                       файл называется полезным, если он реализует полезное требование 
#                                       или разрешает зависимость
# plugins                 (1 x k)     - вектор поставляемых плагинов
# delivery_files          (1 x m)     - вектор с данными о поставляемых файлах исходного кода
# delivery_requirements   (1 x n)     - вектор с данными о поставляемых требованиях
# costs                   (1 x n)     - вектора с стоимостями сопровождения каждого из требований в рамках поставки

In [3]:
import pyomo.environ as pyo
import numpy as np
import pandas as pd

In [4]:
M = 10 ** 6

In [5]:
# Инициализация модели

model = pyo.ConcreteModel(name = 'Optimal decomposition')

In [6]:
# Ограничения модели

model.constraints = pyo.ConstraintList()

In [7]:
# Дополнительные бинарные переменные

model.f = pyo.VarList(domain=pyo.Binary)

In [8]:
check_include_1 = lambda x, f: (f + 1 / M <= x + 1)

In [9]:
check_include_2 = lambda x, f: (x <= M * f)

In [10]:
check_implemented_1 = lambda x, f: (x >= f)

In [11]:
check_implemented_2 = lambda x, f: (x + 1 / M <= 1 + M * f)

In [12]:
# Функция проверки включения
# Пример использования и интерпретация: файл необходимо включить в поставку 
# если на него существует по меньшей мере одна зависимость

def check_include(vector):
    result = []
    for x in vector:
        f = model.f.add()
        result.append(f)
        constraint_1 = check_include_1(x, f)
        constraint_2 = check_include_2(x, f)
        model.constraints.add(constraint_1)
        model.constraints.add(constraint_2)
    return np.array(result)

In [13]:
# Функция проверки реализации
# Пример использования и интерпретация: требование считается реализованным в рамках поставки 
# если в эту поставку включены все файлы исходного кода которые его реализуют

def check_implemented(vector):
    result = []
    for x in vector:
        f = model.f.add()
        result.append(f)
        constraint_1 = check_implemented_1(x, f)
        constraint_2 = check_implemented_2(x, f)
        model.constraints.add(constraint_1)
        model.constraints.add(constraint_2)
    return np.array(result)

In [14]:
# Функция не принимает участие непосредственно в математической моделе.
# Она необходима для демонстрации работы функций проверок включения и реализации.
# Создает таблицу с информацией о принимаемых значениях новосоздаваемой бинарной переменной 
# в зависимости от значения текущего элемента вектора

def test_check(check_function_1, check_function_2):
    x_values_list = []
    f_values_list = []
    check1_and_check2_list = []
    for x in [0, 0.5, 1, 1.5]:   # Значения, которые может принимать вектор
        for f in [0, 1]:         # Значения создаваемой бинарной переменной
            check1_result = check_function_1(x, f)
            check2_result = check_function_2(x, f)
            check1_and_check2_result = check1_result and check2_result
            appended_x_value = x
            if x == 0.5:
                appended_x_value = '(0; 1)'
            if x == 1.5:
                appended_x_value = '(1; ∞)'
            x_values_list.append(appended_x_value)
            f_values_list.append(f)
            check1_and_check2_list.append(check1_and_check2_result)
            
    data = {
        "x": x_values_list,
        "f": f_values_list,
        "valid": check1_and_check2_list
    }
    df = pd.DataFrame(data)
    return df

In [15]:
# Демонстрация работы функций include
# return 0 if x == 0 else 1

test_check(check_include_1, check_include_2)

,x,f,valid
0,0,0,True
1,0,1,False
2,(0; 1),0,False
3,(0; 1),1,True
4,1,0,False
5,1,1,True
6,(1; ∞),0,False
7,(1; ∞),1,True


In [16]:
# Демонстрация работы функций implemented
# return 1 if x >= 1 else 0

test_check(check_implemented_1, check_implemented_2)

,x,f,valid
0,0,0,True
1,0,1,False
2,(0; 1),0,True
3,(0; 1),1,False
4,1,0,False
5,1,1,True
6,(1; ∞),0,False
7,(1; ∞),1,True


In [17]:
# Рекурсивная функция рассчета зависимостей у файлов исходного кода

def calculate_dependencies(i, useful_files):
    if i == 0:
        return np.dot(useful_files, D)
    else:
        dependencies = calculate_dependencies(i - 1, useful_files)
        return np.dot(dependencies, D)

In [18]:
# Матрица трассируемости требований к ПО на файлы исходного кода

T = np.array([
    [  1,   0,   0,   0],  # Требование № 1 реализовано в файле № 1
    [0.5, 0.5,   0,   0],  # Требование № 2 реализовано в файлах № 1 и 2
    [  0, 0.5, 0.5,   0],  # Требования № 3 реализовано в файлах № 2 и 3
    [  0,   0, 0.5, 0.5],  # Требование № 4 реализовано в файлах № 3 и 4
    [  0,   0,   0,   1]
]) # Требовнаие № 5 реализовано в файле № 4

In [19]:
# Матрица зависимостей между файлами исходного кода

D = np.array([
    [0, 1, 0, 0],  # Файл № 1 имеет зависимость на файл № 2
    [0, 0, 0, 0],  # Файл № 2 не имеет зависимостей на другие файлы
    [0, 0, 0, 0],  # Файл № 3 не имеет зависимостей на другие файлы
    [0, 0, 0, 0]
]) # Файл № 4 не имеет зависимостей на другие файлы

In [20]:
# Матрица стоимостей вхождения требований в поставку

P = np.array([
    [1, 0, 0, 0, 0], # Стоимость сопровождения функционала, если в поставку войдет требование № 1
    [0, 1, 0, 0, 0], # Стоимость сопровождения функционала, если в поставку войдет требование № 2
    [0, 0, 1, 0, 0], # Стоимость сопровождения функционала, если в поставку войдет требование № 3
    [0, 0, 0, 1, 0], # Стоимость сопровождения функционала, если в поставку войдет требование № 4
    [0, 0, 0, 0, 1]  # Стоимость сопровождения функционала, если в поставку войдет требование № 5
])

In [21]:
# Число плагинов

k = 3

In [22]:
# Число требований(n) и число файлов исходного кода(m)

n, m = np.shape(T)

In [23]:
# Матрица распределения файлов по плагинам
# Необходимо определить оптимальное распределение
# Здесь указаны ограничения на максимальное и минимальное значение
# И на то, что элементы должны быть целыми числами

set_m = pyo.Set(initialize=range(m))
set_k = pyo.Set(initialize=range(k))
model.A = pyo.Var(set_m, set_k, domain=pyo.Binary)

# A = np.array([
#     [1, 0, 0],
#     [0, 1, 0],
#     [0, 1, 0],
#     [0, 0, 1]
# ])


In [24]:
# На искомые элементы накладываются ограничения:
# сумма элементов в каждой строке матрицы равна 1

for row in np.array(model.A):
    model.constraints.add((sum(row) == 1))

In [25]:
# Полезные требования

useful_requirements = np.array([1, 0, 0, 0, 0])

In [26]:
# Полезные файлы исходного кода

useful_files = np.dot(useful_requirements, T)

In [27]:
# Рассчет заивисимостей между файлами исходного кода

for i in range(len(useful_files)):
    dependencies = calculate_dependencies(i, useful_files)
    useful_files += dependencies

In [28]:
# Поставляемые плагины

plugins = np.dot(useful_files, model.A)

In [29]:
plugins = check_include(plugins)

In [30]:
# Поставляемые файлы исходного кода

delivery_files = np.dot(model.A, plugins)

In [31]:
delivery_files = check_include(delivery_files)

In [32]:
# Поставляемые реализованные требования

delivery_requirements = np.dot(T, delivery_files)

In [33]:
delivery_requirements = check_implemented(delivery_requirements)

In [34]:
# Затраты на сопровождение требований в поставке

costs = np.dot(P, delivery_requirements)

In [35]:
# Целевая функция

model.OBJ = pyo.Objective(expr = sum(costs), sense=pyo.minimize)

In [36]:
model.pprint()

2 Var Declarations
    A : Size=12, Index={0, 1, 2, 3}*{0, 1, 2}
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (0, 0) :     0 :  None :     1 : False :  True : Binary
        (0, 1) :     0 :  None :     1 : False :  True : Binary
        (0, 2) :     0 :  None :     1 : False :  True : Binary
        (1, 0) :     0 :  None :     1 : False :  True : Binary
        (1, 1) :     0 :  None :     1 : False :  True : Binary
        (1, 2) :     0 :  None :     1 : False :  True : Binary
        (2, 0) :     0 :  None :     1 : False :  True : Binary
        (2, 1) :     0 :  None :     1 : False :  True : Binary
        (2, 2) :     0 :  None :     1 : False :  True : Binary
        (3, 0) :     0 :  None :     1 : False :  True : Binary
        (3, 1) :     0 :  None :     1 : False :  True : Binary
        (3, 2) :     0 :  None :     1 : False :  True : Binary
    f : Size=12, Index={1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
        Key : Lower : Value : Upper : Fi

In [37]:
def solve(solver_name):
    solver = pyo.SolverFactory(solver_name)
    instance = model.create_instance()
    result = solver.solve(instance)
    instance.A.display()

In [38]:
solve('gams')

model.name="Optimal decomposition";
    - termination condition: intermediateNonInteger
    - message from solver: Solver quit with a problem (see LST file)
A : Size=12, Index={0, 1, 2, 3}*{0, 1, 2}
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (0, 0) :     0 :   1.0 :     1 : False : False : Binary
    (0, 1) :     0 :   0.0 :     1 : False : False : Binary
    (0, 2) :     0 :   0.0 :     1 : False : False : Binary
    (1, 0) :     0 :   1.0 :     1 : False : False : Binary
    (1, 1) :     0 :   0.0 :     1 : False : False : Binary
    (1, 2) :     0 :   0.0 :     1 : False : False : Binary
    (2, 0) :     0 :   0.0 :     1 : False : False : Binary
    (2, 1) :     0 :   1.0 :     1 : False : False : Binary
    (2, 2) :     0 :   0.0 :     1 : False : False : Binary
    (3, 0) :     0 :   1.0 :     1 : False : False : Binary
    (3, 1) :     0 :   0.0 :     1 : False : False : Binary
    (3, 2) :     0 :   0.0 :     1 : False : False : Binary


In [39]:
solve('ipopt')

A : Size=12, Index={0, 1, 2, 3}*{0, 1, 2}
    Key    : Lower : Value               : Upper : Fixed : Stale : Domain
    (0, 0) :     0 :  0.3333335289569246 :     1 : False : False : Binary
    (0, 1) :     0 : 0.33333319677321593 :     1 : False : False : Binary
    (0, 2) :     0 :  0.3333332742698596 :     1 : False : False : Binary
    (1, 0) :     0 : 0.33333330469243017 :     1 : False : False : Binary
    (1, 1) :     0 :  0.3333333649692199 :     1 : False : False : Binary
    (1, 2) :     0 :    0.33333333033835 :     1 : False : False : Binary
    (2, 0) :     0 :   0.333333325187196 :     1 : False : False : Binary
    (2, 1) :     0 :  0.3333333457906409 :     1 : False : False : Binary
    (2, 2) :     0 : 0.33333332902216317 :     1 : False : False : Binary
    (3, 0) :     0 : 0.33333332505579394 :     1 : False : False : Binary
    (3, 1) :     0 :  0.3333333459842568 :     1 : False : False : Binary
    (3, 2) :     0 :  0.3333333289599494 :     1 : False : False : Bin

In [40]:
solve('mpec_nlp')

A : Size=12, Index={0, 1, 2, 3}*{0, 1, 2}
    Key    : Lower : Value               : Upper : Fixed : Stale : Domain
    (0, 0) :     0 :  0.3333335289569246 :     1 : False : False : Binary
    (0, 1) :     0 : 0.33333319677321593 :     1 : False : False : Binary
    (0, 2) :     0 :  0.3333332742698596 :     1 : False : False : Binary
    (1, 0) :     0 : 0.33333330469243017 :     1 : False : False : Binary
    (1, 1) :     0 :  0.3333333649692199 :     1 : False : False : Binary
    (1, 2) :     0 :    0.33333333033835 :     1 : False : False : Binary
    (2, 0) :     0 :   0.333333325187196 :     1 : False : False : Binary
    (2, 1) :     0 :  0.3333333457906409 :     1 : False : False : Binary
    (2, 2) :     0 : 0.33333332902216317 :     1 : False : False : Binary
    (3, 0) :     0 : 0.33333332505579394 :     1 : False : False : Binary
    (3, 1) :     0 :  0.3333333459842568 :     1 : False : False : Binary
    (3, 2) :     0 :  0.3333333289599494 :     1 : False : False : Bin

In [41]:
solve('multistart')

A : Size=12, Index={0, 1, 2, 3}*{0, 1, 2}
    Key    : Lower : Value               : Upper : Fixed : Stale : Domain
    (0, 0) :     0 : 0.33333332910276614 :     1 : False :  True : Binary
    (0, 1) :     0 : 0.33333334149775323 :     1 : False :  True : Binary
    (0, 2) :     0 :  0.3333333293994807 :     1 : False :  True : Binary
    (1, 0) :     0 : 0.33333332717203695 :     1 : False :  True : Binary
    (1, 1) :     0 :  0.3333333428824831 :     1 : False :  True : Binary
    (1, 2) :     0 : 0.33333332994547993 :     1 : False :  True : Binary
    (2, 0) :     0 :  0.3333333294799998 :     1 : False :  True : Binary
    (2, 1) :     0 : 0.33333333910204727 :     1 : False :  True : Binary
    (2, 2) :     0 :   0.333333331417953 :     1 : False :  True : Binary
    (3, 0) :     0 :  0.3333333293941128 :     1 : False :  True : Binary
    (3, 1) :     0 :  0.3333333392306132 :     1 : False :  True : Binary
    (3, 2) :     0 :   0.333333331375274 :     1 : False :  True : Bin

In [42]:
solve('ipopt_v2')

A : Size=12, Index={0, 1, 2, 3}*{0, 1, 2}
    Key    : Lower : Value               : Upper : Fixed : Stale : Domain
    (0, 0) :     0 :  0.3333335289569246 :     1 : False : False : Binary
    (0, 1) :     0 : 0.33333319677321593 :     1 : False : False : Binary
    (0, 2) :     0 :  0.3333332742698596 :     1 : False : False : Binary
    (1, 0) :     0 : 0.33333330469243017 :     1 : False : False : Binary
    (1, 1) :     0 :  0.3333333649692199 :     1 : False : False : Binary
    (1, 2) :     0 :    0.33333333033835 :     1 : False : False : Binary
    (2, 0) :     0 :   0.333333325187196 :     1 : False : False : Binary
    (2, 1) :     0 :  0.3333333457906409 :     1 : False : False : Binary
    (2, 2) :     0 : 0.33333332902216317 :     1 : False : False : Binary
    (3, 0) :     0 : 0.33333332505579394 :     1 : False : False : Binary
    (3, 1) :     0 :  0.3333333459842568 :     1 : False : False : Binary
    (3, 2) :     0 :  0.3333333289599494 :     1 : False : False : Bin